#Merging, Cleaning Data Sets and Adding Variables#

This notebook will be used to explore merging the data sets, cleaning the data sets (changing various values to np.NaN, converting fields to the appropriate value type), and adding calculated variables. The cleaned data st is exported as csv to be used in other data analysis.

###Contents###

1. Merging Data Sets
2. Cleaning Data Sets
    - recoding null values to np.NaN
    - correcting data types
    - adding calculated variables (outcome measures, avg disc score, etc)
3. Dealing with Null Values
4. Adding new calculated fields (CVD history, education, income)
5. Replacing category values with names (i.e. race)
6. Writing to CSV file

In [1]:
import pandas as pd
import numpy as np

cross = pd.read_csv('001_CrossSectional_data.csv', low_memory=False)
base = pd.read_csv('00_Baseline.csv', low_memory=False)

pd.set_option('display.max_rows', 120)
cross.rename(columns={'ID':'SWANID'}, inplace=True)

##Merging Data Sets##

In [2]:
data = pd.merge(cross, base)

In [3]:
data.shape

(3302, 858)

That's a lot of fields.

##Cleaning Data Sets##

###Changing coded values for missing/null data to np.NaN###

In [4]:
data.replace(' ', np.nan, inplace=True)
data.replace('-9', np.nan, inplace=True)
data.replace('-1', np.nan, inplace=True)
data.replace('-7', np.nan, inplace=True)
data.replace('-8', np.nan, inplace=True)

###Correcting data types###

Discrimination scores

In [5]:
data[['COURTES0', 'RESPECT0', 'POORSER0', 'NOTSMAR0', 'AFRAIDO0', 'DISHONS0', 'BETTER0', 'INSULTE0', 
      'HARASSE0', 'IGNORED0']] = data[['COURTES0', 'RESPECT0', 'POORSER0', 'NOTSMAR0', 'AFRAIDO0', 'DISHONS0', 
                                       'BETTER0', 'INSULTE0', 'HARASSE0', 'IGNORED0']].astype(float)

Ages, BMI, P_STRESS, CRPRESU0

In [6]:
data[['AGE0']] = data[['AGE0']].astype(float)
data[['BMI0']] = data[['BMI0']].astype(float)
data[['P_STRESS']] = data[['P_STRESS']].astype(float)
data[['CRPRESU0']] = data[['CRPRESU0']].astype(float)
data[['HEART']] = data[['HEART']].astype(float)
data[['STROKE0']] = data[['STROKE0']].astype(float)
data[['INCOME0']] = data[['INCOME0']].astype(float)
data[['DEGREE']] = data[['DEGREE']].astype(float)
data[['ETHNIC']] = data[['ETHNIC']].astype(float)

###Adding Calculated Variables###

In [7]:
def convert_binary(cols):
    data[cols] = data[cols].map({'1':0, '2':1})
    # data[cols].replace(np.nan, 0, inplace=True)
    print data[cols].value_counts(dropna=False)

def convert_binary_float(cols):
    data[cols] = data[cols].map({1:0, 2:1})
    # data[cols].replace(np.nan, 0, inplace=True) # this may need to come out for certain variables
    print data[cols].value_counts(dropna=False)

In [8]:
data.STROKE0.replace(1, 0, inplace=True)
data.STROKE0.replace(2, 1, inplace=True)

data.HEART.replace(1, 0, inplace=True)
data.HEART.replace(2, 1, inplace=True)

*Average Discrimination Score*

In [9]:
data.loc[:, 'DISC_SCORE0'] = 5 - data[['COURTES0', 'RESPECT0', 'POORSER0', 'NOTSMAR0', 'AFRAIDO0', 'DISHONS0', 
                                   'BETTER0', 'INSULTE0', 'HARASSE0', 'IGNORED0']].mean(axis=1)

*Reason for Discrimination*

In [10]:
data['RACE_REASON0'] = data.MAINREA0.map({'1':1, '2':1, '3':0, '4':0, '5':0, '6':0, '7':0, '8':0, '9':0})

*Outcome measures*

In [11]:
# Difference between Lipid Vascular Age and actual age

data['LV_AGE_DIFF0'] = data.LV_AGE0 - data.AGE0

# Difference between BMI Vascular Age and actual age

data['BV_AGE_DIFF0'] = data.BV_AGE0 - data.AGE0


# % change between Lipid Vascular Age and actual age

data['LV_AGE_PCT0'] = (data.LV_AGE0 - data.AGE0) / data.AGE0

# % change between BMI Vascular Age and actual age

data['BV_AGE_PCT0'] = (data.BV_AGE0 - data.AGE0) / data.AGE0


##Dealing with Null Values##

1) Remove all records that are missing either a discrimination score or a lipid vascular age

In [12]:
data.shape

(3302, 864)

In [13]:
data.DISC_SCORE0.isnull().sum()

6

In [14]:
data.LV_AGE0.isnull().sum()

35

In [15]:
data.dropna(subset=['DISC_SCORE0', 'LV_AGE0'], inplace=True)

In [16]:
data.shape

(3262, 864)

2) Replacing null values with average for continuous variables (BMI, CRPRESU0)

In [17]:
#data.dropna(subset=['BMI0', 'INCOME0', 'DEGREE', 'CRPRESU0', 'P_STRESS', 'HEART', 'STROKE0'], inplace=True)

data.BMI0.isnull().sum()

42

In [18]:
data.CRPRESU0.isnull().sum()

36

In [19]:
data.BMI0.mean()

28.254157491155254

In [20]:
data.CRPRESU0.mean()

3.8906642901425808

In [21]:
# replace depending on race
# means = data.groupby('ETHNIC').P_STRESS.mean()
# for i in data[data.P_STRESS.isnull()].index:
#    df.loc[i, 'P_STRESS'] = means[df.loc[i].ETHNIC]

# replace with the average value for all null values

data.BMI0.fillna(data.BMI0.mean(), inplace=True)
data.CRPRESU0.fillna(data.CRPRESU0.mean(), inplace=True)

In [22]:
data.MOD_PSTRESS.isnull().sum()

54

In [23]:
data.MOD_PSTRESS.replace(np.nan, 8, inplace=True)

3) Check null values for Income, Degree, Heart, and Stroke0

In [24]:
data.INCOME0.isnull().sum()

84

In [25]:
data.DEGREE.isnull().sum()

31

In [26]:
data.HEART.isnull().sum()

53

In [27]:
data.STROKE0.isnull().sum()

2

4) Replacing the null values for Income, Degree, Heart, and Stroke0, with the most frequent value

In [28]:
data.INCOME0.replace(np.nan, 3, inplace=True)

In [29]:
data.DEGREE.replace(np.nan, 3, inplace=True)

In [30]:
data.HEART.replace(np.nan, 0, inplace=True)

In [31]:
data.STROKE0.replace(np.nan, 0, inplace=True)

##Adding more Calculated Fields##

**Adding the CVD History field**

In [32]:
data['CVD_COMBINE'] = data.STROKE0 + data.HEART
data['CVD'] = np.where(data['CVD_COMBINE'] > 0, 1, 0)

**Adding New Education field**

Combines to find # of participants that have higher than a high school education

In [33]:
data['NEWEDU'] = np.where(data['DEGREE'] < 3, 0, 1)

**Adding New Income field**

Combines to find the # of participants that have higher than 50K in annual income

In [34]:
data['NEWINCOME'] = np.where(data['INCOME0'] < 3, 0, 1)

##Replacing category numbers with names##

Race

In [35]:
data['RACE'] = data.ETHNIC.map({1:'Black', 8:'Asian', 9:'Asian', 10:'Caucasian', 13:'Hispanic'})

In [36]:
data.RACE.value_counts(dropna=False)

Caucasian    1529
Black         922
Asian         529
Hispanic      282
Name: RACE, dtype: int64

BMI (putting into categories)

In [37]:
data['BMI_CAT'] = data.BMI0

data.loc[data.BMI_CAT < 18, ['BMI_CAT']] = '0_Underweight'
data.loc[data.BMI_CAT < 25, ['BMI_CAT']] = '1_HealthyWeight'
data.loc[data.BMI_CAT < 30, ['BMI_CAT']] = '2_Overweight'
data.loc[data.BMI_CAT < 100, ['BMI_CAT']] = '3_Obese'

##Writing to a new CSV##

In [38]:
data.to_csv('991_CleanedData.csv', index = False)

In [39]:
data.shape

(3262, 870)